In [12]:
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [20]:
def es_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    es_client = Elasticsearch('http://localhost:9200')
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [21]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. 
        Answer the QUESTION based on the CONTEXT. 
        Use only the facts from the CONTEXT when answering the QUESTION.
        If the CONTEXT doesn't contain the answer, output IDK
        Answer in at most three sentences.
        
        QUESTION: {question}
        
        CONTEXT: 
        {context}
        
    """.strip()
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [22]:
def llm(prompt):

    client = OpenAI(
        base_url='http://localhost:11434/v1/',
        api_key='ollama',
    )
      
    response = client.chat.completions.create(
        model = 'phi3',
        messages = [{"role":"user", "content":prompt}]
    )

    return response.choices[0].message.content

In [23]:
def rag(query):
    search_results = es_search(query)
    prompt = build_prompt(query, search_results)
    return llm(prompt)

In [24]:
query = 'if the course already started, may I still join?'

In [25]:
print(rag(query))

 Yes, you can still join the course even if it has already started; however, you won't be able to submit some of the homeworks but will be able to participate and work on projects.
